In [1]:
!pip show torchvision
!pip install torchvision==0.16.1
!python3 -m venv myenv
!source myenv/bin/activate
!pip install --upgrade pip setuptools
!pip uninstall -y transformers sentence-transformers
!pip install transformers==4.37.0 sentence-transformers torch torchvision torchaudio
!pip install --upgrade "tf-keras<2.16.0"
!python -c "import torch; print(torch.__version__)"


Name: torchvision
Version: 0.16.1
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /opt/miniconda3/lib/python3.10/site-packages
Requires: numpy, pillow, requests, torch
Required-by: 
Found existing installation: transformers 4.43.0
Uninstalling transformers-4.43.0:
  Successfully uninstalled transformers-4.43.0
Found existing installation: sentence-transformers 3.0.1
Uninstalling sentence-transformers-3.0.1:
  Successfully uninstalled sentence-transformers-3.0.1
  Using cached transformers-4.37.0-py3-none-any.whl.metadata (129 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13

In [2]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())  # Should return True if MPS is available

2.1.1
True


In [3]:
pip install transformers==4.43.0

  Using cached transformers-4.43.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached transformers-4.43.0-py3-none-any.whl (9.4 MB)
Using cached tokenizers-0.19.1-cp310-cp310-macosx_11_0_arm64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-4.37.0━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]
Note: you may need to restart the kernel to use updated packages.


In [4]:
import transformers
print(transformers.__version__)

4.43.0


In [5]:
import pandas as pd
import torch
import numpy as np
import re
import random
from collections import Counter
from fuzzywuzzy import fuzz
from scipy.stats import entropy
from transformers import AutoModelForCausalLM, AutoTokenizer
import matplotlib.pyplot as plt

# Check for available device
device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/opt/miniconda3/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Using device: mps


In [6]:
# Load LLM model and tokenizer
model_name = "microsoft/Phi-3.5-mini-instruct"
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, max_memory={device: "10GB"}, trust_remote_code=True, force_download=True).to(device)
print("Model and tokenizer loaded successfully.")


Loading model and tokenizer...


config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


In [7]:
data_path = "/Users/tomi/Desktop/Thesis/Data/"

# Load data
print("Loading dataset...")
test_data_lastFM1k_fullInteraction= pd.read_csv(data_path + "test_data_lastFM1k_fullInteraction_80users.csv")
print("Dataset loaded successfully.")

print("\nFirst 10 rows of test_data_lastFM1k_fullInteraction:")
print(test_data_lastFM1k_fullInteraction.head(10))

Loading dataset...
Dataset loaded successfully.

First 10 rows of test_data_lastFM1k_fullInteraction:
   Unnamed: 0         user                  timestamp        artist  \
0           0  user_000007  2007-08-10 03:39:32+00:00     Bob James   
1           1  user_000007  2007-08-10 03:45:21+00:00   Boney James   
2           2  user_000007  2007-08-10 04:05:18+00:00     Shakedown   
3           3  user_000007  2007-08-10 04:08:16+00:00          Silk   
4           4  user_000007  2007-08-10 04:13:30+00:00  Pretty Ricky   
5           5  user_000007  2007-08-10 04:16:48+00:00          Akon   
6           6  user_000007  2007-08-10 04:24:23+00:00        T-Pain   
7           7  user_000007  2007-08-11 02:15:32+00:00          Akon   
8           8  user_000007  2007-08-11 02:30:08+00:00  Pretty Ricky   
9           9  user_000007  2007-08-11 02:34:36+00:00          Akon   

                               track_id  \
0  212e46a0-de16-4c92-9b00-f4d9f506c5a6   
1  e873111e-e1a7-4885-8d6a-911

In [8]:
print(model)  # Should print the model architecture, not None
print(tokenizer)  # Should print tokenizer details

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [9]:
# Function to normalize music names
#def normalize_list(lst):
#    return [re.sub(r'[^a-zA-Z0-9 ]', '', s.lower().strip()) for s in lst]

In [14]:
# Evaluation Metrics

def normalize_titles(titles):
    return [t.strip().lower() for t in titles]

def hit_rate(recommendations, ground_truth, top_k=3, threshold=60):
    recommendations = normalize_titles(recommendations)
    ground_truth = normalize_titles(ground_truth)

    hits = sum(
        any(fuzz.partial_ratio(rec, truth) >= threshold for truth in ground_truth)
        for rec in recommendations[:top_k]
    )
    return hits / top_k if top_k > 0 else 0

def average_rank(recommendations, ground_truth):
    recommendations = normalize_titles(recommendations)
    ground_truth = normalize_titles(ground_truth)
    
    ranks = [
        next((i + 1 for i, rec in enumerate(recommendations) if rec == truth), None)
        for truth in ground_truth
    ]
    ranks = [r for r in ranks if r is not None]
    return np.mean(ranks) if ranks else np.nan

def recall_at_k(preds, truths, k=3):
    preds = normalize_titles(preds[:k])
    truths = normalize_titles(truths)
    hits = len(set(preds) & set(truths))
    return hits / len(set(truths)) if truths else 0.0

def dcg_at_k(recs, truths, k):
    recs = normalize_titles(recs[:k])
    truths = set(normalize_titles(truths))
    return sum(1 / np.log2(i + 2) for i, rec in enumerate(recs) if rec in truths)

def ndcg_at_k(recs, truths, k):
    ideal_dcg = dcg_at_k(truths, truths, k)
    actual_dcg = dcg_at_k(recs, truths, k)
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0.0

def hhi(recommendations):
    counter = Counter(recommendations)
    total = sum(counter.values())
    return sum((count / total) ** 2 for count in counter.values()) if total > 0 else 0

def entropy(recommendations):
    counter = Counter(recommendations)
    total = sum(counter.values())
    return -sum((count / total) * np.log2(count / total) for count in counter.values() if count > 0) if total > 0 else 0

def gini_index(recommendations):
    if not recommendations:
        return 0
    values = np.array(list(Counter(recommendations).values()))
    values = values / values.sum()
    values.sort()
    n = len(values)
    index = np.arange(1, n + 1)
    return (2 * np.sum(index * values) - (n + 1) * np.sum(values)) / (n * np.sum(values))

def gini_index_scores(x):
    x = np.array(x)
    if np.amin(x) < 0:
        x -= np.amin(x)
    x += 1e-6
    x_sorted = np.sort(x)
    n = len(x)
    cumx = np.cumsum(x_sorted)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

### Diversify model

In [23]:
def get_recommendations_s3_music_diversify(user_history, candidate_tracks_df, k=10):
    print(f"Generating S3 Diversify recommendations for user {user_history['userId'].iloc[0]}...")

    # 1) Artist weights from user history
    artist_scores = user_history['artist'].value_counts().to_dict()

    # 2) Score candidates
    results = []
    for idx, r in candidate_tracks_df.iterrows():
        artist_score = artist_scores.get(r['artist'], 0)
        pop = r.get('normalized_popularity', 0)
        noise = np.random.normal(0, 0.5)
        bias = idx * 1e-4
        score = artist_score + 0.7 * pop + noise + bias
        results.append((r['track_name'], score))

    # 3) Top-k
    topk = sorted(results, key=lambda x: x[1], reverse=True)[:k]
    titles, raw_scores = zip(*topk)

    # 4) Normalize for diversity
    raw_scores = np.array(raw_scores)
    if np.std(raw_scores) < 1e-3:
        raw_scores += np.random.normal(0, 0.1, size=len(raw_scores))
    raw_scores -= raw_scores.min()
    raw_scores += 1e-6
    scores = raw_scores.tolist()

    # 5) Prompt string
    user_tracks_string = ", ".join(user_history['track_name'].tolist()[:5])
    prompt = (
        f"Provide 10 music recommendations that explore new styles while remaining interesting to the user.\n"
        f"User has recently listened to: {user_tracks_string}.\n"
        f"Aim to balance diversity and relevance in your choices."
    )
    print("Gini input scores:", scores)
    return list(titles), scores, prompt


In [24]:
# Select candidates not in user history
user_ids = test_data_lastFM1k_fullInteraction['userId'].unique()[:10]
metrics_music_s3 = []

for uid in user_ids:
    hist = test_data_lastFM1k_fullInteraction[test_data_lastFM1k_fullInteraction['userId'] == uid]
    if len(hist) < 6:
        continue

    input_history = hist.iloc[:5]
    truth_tracks = hist.iloc[5:]['track_name'].tolist()

    candidate_tracks = test_data_lastFM1k_fullInteraction[
        (~test_data_lastFM1k_fullInteraction['track_id'].isin(input_history['track_id']))
    ].copy()

    # Popularity normalization
    track_counts = test_data_lastFM1k_fullInteraction['track_id'].value_counts()
    candidate_tracks['popularity'] = candidate_tracks['track_id'].map(track_counts).fillna(0)
    candidate_tracks['normalized_popularity'] = candidate_tracks['popularity'] / candidate_tracks['popularity'].max()
    candidate_tracks['normalized_popularity'] += np.random.normal(0, 0.01, size=len(candidate_tracks))

    # Sample for performance
    candidate_tracks = candidate_tracks.sample(min(len(candidate_tracks), 120), random_state=42)

    # Get diversified recommendations
    recs, scores, prompt = get_recommendations_s3_music_diversify(input_history, candidate_tracks)

    print(f"\nUser {uid} Prompt:\n{prompt}\n")

    # Save metrics
    metrics_music_s3.append({
    "hit_rate": hit_rate(recs, truth_tracks, top_k=3),
    "avg_rank": average_rank(recs, truth_tracks),
    "recall@5": recall_at_k(recs, truth_tracks, k=5),
    "ndcg@5": ndcg_at_k(recs, truth_tracks, k=5),
    "hhi": hhi(recs),
    "entropy": entropy(recs),
    "gini": gini_index_scores(scores)
})

# Show summary
df_metrics_music_s3 = pd.DataFrame(metrics_music_s3)
print("\nAverage Metrics (S3 Music Diversify):")
print(df_metrics_music_s3.mean())

Generating S3 Diversify recommendations for user 6...
Gini input scores: [0.72918383317878, 0.4742127913621495, 0.45502301045872945, 0.3413406267472164, 0.307062161026362, 0.18689573085456232, 0.18611667309511834, 0.1706734613342292, 0.018298249905392833, 1e-06]

User 6 Prompt:
Provide 10 music recommendations that explore new styles while remaining interesting to the user.
User has recently listened to: Take Me There, Sara Smile, At Night (Afterlife Mix), Lose Control, Push It Baby (Radio Version).
Aim to balance diversity and relevance in your choices.

Generating S3 Diversify recommendations for user 46...
Gini input scores: [0.9657059650001062, 0.7474965249744415, 0.5780604690338337, 0.5363446670723374, 0.5327137605298836, 0.5313529685152368, 0.3085669636259077, 0.24771116362088805, 0.003911915057955045, 1e-06]

User 46 Prompt:
Provide 10 music recommendations that explore new styles while remaining interesting to the user.
User has recently listened to: Odmah Kaži Da, Okrutni Bogo

### Chain of Thought (COT)

In [30]:
from fuzzywuzzy import fuzz

def normalize_titles(titles):
    return [t.strip().lower() for t in titles]

def average_rank(recommendations, ground_truth, threshold=85):
    recs = normalize_titles(recommendations)
    truths = normalize_titles(ground_truth)
    ranks = [
        idx + 1
        for truth in truths
        for idx, rec in enumerate(recs)
        if fuzz.partial_ratio(rec, truth) >= threshold
    ]
    return sum(ranks) / len(ranks) if ranks else np.nan

def recall_at_k(preds, truths, k=5, threshold=85):
    preds_k = normalize_titles(preds[:k])
    truths_norm = normalize_titles(truths)
    hits = sum(any(fuzz.partial_ratio(p, t) >= threshold for t in truths_norm) for p in preds_k)
    return hits / len(set(truths_norm)) if truths_norm else 0.0

def dcg_at_k(recs, truths, k=5, threshold=85):
    recs_k = normalize_titles(recs[:k])
    truths_norm = normalize_titles(truths)
    return sum(
        1 / np.log2(i + 2)
        for i, rec in enumerate(recs_k)
        if any(fuzz.partial_ratio(rec, t) >= threshold for t in truths_norm)
    )

def ndcg_at_k(recs, truths, k=5, threshold=85):
    ideal_dcg = dcg_at_k(truths, truths, k, threshold)
    actual_dcg = dcg_at_k(recs, truths, k, threshold)
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0.0


In [31]:
from fuzzywuzzy import fuzz
from collections import defaultdict
import numpy as np
import pandas as pd
import torch

# --- Gini Index ---
def gini_index_scores(x):
    x = np.array(x)
    if x.size == 0:
        return 0.0
    if np.amin(x) < 0:
        x -= np.amin(x)
    x += 1e-6
    x_sorted = np.sort(x)
    n = len(x)
    cumx = np.cumsum(x_sorted)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

# --- Fuzzy filter ---
def fuzzy_filter_titles(recommended_titles, valid_titles, threshold=85):
    filtered = []
    for rec in recommended_titles:
        best_match = max(valid_titles, key=lambda v: fuzz.ratio(v.lower(), rec.lower()), default=None)
        if best_match and fuzz.ratio(best_match.lower(), rec.lower()) >= threshold:
            filtered.append(best_match)
    return list(dict.fromkeys(filtered))  # dedup while preserving order

# --- LLM-based COT Recommendation ---
def get_recommendations_s7_music_cot(user_history, candidate_df, tokenizer, model, device, k=10):
    listened_tracks = user_history['track_name'].tolist()
    user_tracks_str = ", ".join(listened_tracks[:5])

    prompt = (
        f"The user has listened to the following tracks: {user_tracks_str}. "
        f"Please recommend {k} real track titles that match their taste. "
        f"List them clearly like: 1. Track Title - Reason"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.9,
            do_sample=True,
            top_p=0.95
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Parse recommendations
    lines = decoded.split("\n")
    recommended_titles = []
    for line in lines:
        if "." in line:
            try:
                title_reason = line.split(".", 1)[1].strip()
                if "-" in title_reason:
                    title = title_reason.split("-", 1)[0].strip()
                    recommended_titles.append(title)
            except IndexError:
                continue

    valid_titles = candidate_df['track_name'].tolist()
    filtered_titles = fuzzy_filter_titles(recommended_titles, valid_titles)
    return filtered_titles[:k], decoded


In [32]:
user_ids = test_data_lastFM1k_fullInteraction['userId'].unique()[:10]
metrics_s7_music = []
all_recommendations_s7_music = []
item_exposure_scores_music = defaultdict(float)

for uid in user_ids:
    hist = test_data_lastFM1k_fullInteraction[test_data_lastFM1k_fullInteraction['userId'] == uid]
    if len(hist) < 6:
        continue

    input_history = hist.iloc[:5]
    truth_tracks = hist.iloc[5:]['track_name'].tolist()
    candidate_tracks = test_data_lastFM1k_fullInteraction[
        ~test_data_lastFM1k_fullInteraction['track_id'].isin(input_history['track_id'])
    ].copy()

    # Normalize popularity
    track_popularity = test_data_lastFM1k_fullInteraction['track_id'].value_counts()
    candidate_tracks['popularity'] = candidate_tracks['track_id'].map(track_popularity).fillna(0)
    candidate_tracks['normalized_popularity'] = candidate_tracks['popularity'] / candidate_tracks['popularity'].max()

    rec_titles, full_response = get_recommendations_s7_music_cot(
        input_history, candidate_tracks, tokenizer, model, device
    )

    print(f"\nUser {uid}")
    print(f"Ground truth: {truth_tracks}")
    print(f"Recommendations: {rec_titles}")
    print(f"LLM Response:\n{full_response}")

    if not rec_titles:
        print(f"Skipping user {uid} due to empty recommendations.\n")
        continue

    # Exposure score per rank
    for i, title in enumerate(rec_titles):
        score = 1.0 - 0.05 * i
        item_exposure_scores_music[title] += score

    all_recommendations_s7_music.extend(rec_titles)

    metrics_s7_music.append({
        "hit_rate": hit_rate(rec_titles, truth_tracks),
        "avg_rank": average_rank(rec_titles, truth_tracks),
        "recall@5": recall_at_k(rec_titles, truth_tracks, k=5),
        "ndcg@5": ndcg_at_k(rec_titles, truth_tracks, k=5),
        "hhi": hhi(rec_titles),
        "entropy": entropy(rec_titles)
    })


# --- Summary ---
df_metrics_s7_music = pd.DataFrame(metrics_s7_music)
print("\nAverage Metrics for First 10 Users (S7 COT Music):")
print(df_metrics_s7_music.mean(numeric_only=True))

print("\nSystem-Level Entropy Across All S7 Music Recommendations:")
print(entropy(all_recommendations_s7_music))

print("\nSystem-Level Gini Index Across All S7 Music Recommendations:")
gini_scores_music = list(item_exposure_scores_music.values())
if gini_scores_music:
    print(gini_index_scores(gini_scores_music))
else:
    print("Gini Index: N/A (no exposure scores recorded)")


User 6
Ground truth: ["I Can'T Wait", 'Show U How (Feat. Teddy Pain & Teddy Penderazdoun)', "I Can'T Wait", 'Push It Baby (Radio Version)', "I Can'T Wait", 'Trouble Sleeping', 'Welcome To Jamrock', 'Superman', 'No Guarantee Remix (Feat. Joe)', "Pullin' Me Back", 'Come Around', "Ascension (Don'T Ever Wonder)", 'Kissing A Fool', 'Miss Fatty', "I Can'T Wait", 'Certified', 'Get It Shawty', 'Wonderful', 'All That I Can Say', "Signs Of Love Makin'", 'Stop, Look, Listen To Your Heart', 'Wishing On A Star', 'Push It Baby (Radio Version)', "I Can'T Wait", "I Can'T Wait", 'Superman', "Pullin' Me Back", 'Certified', 'Wonderful', "Ascension (Don'T Ever Wonder)", 'Kissing A Fool', 'Push It Baby (Radio Version)', 'Everyday', 'Deep Waters', "Talkin' 'Bout Love", 'Take Me There', 'Sara Smile', 'Si Tu Veux De Moi', 'Enchantment', 'Last Night', 'Sit Down', 'Pop Pop', 'Wait For Me', 'If I Was Your Man', 'Circle', 'Not Like This', 'Ask Of You', 'Get It Shawty', 'Cross My Mind', "Be Here (Feat. D'Angelo)"